# Route Recommendation Visualization

This notebook contains functions to:
1. Take route IDs from recommendations
2. Load route JSON from GCS
3. Display routes on interactive maps

To be integrated into the Streamlit app later.

## Setup and Imports

In [2]:
import os
import json
from typing import List, Tuple, Dict, Any
import folium
from google.cloud import storage
import fsspec

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Verify GCP credentials are loaded
gcp_creds = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
print(f"GCP Credentials path: {gcp_creds}")
print(f"Credentials file exists: {os.path.exists(gcp_creds) if gcp_creds else False}")

GCP Credentials path: /Users/henryberkes/code/henrybrk/gcp/cyclemore-342ee28c6cc7.json
Credentials file exists: True


## Function 1: Load Route JSON from GCS by ID

In [3]:
def find_route_in_gcs(bucket_name: str, route_id: int, prefix: str = "all_routes/") -> str:
    """
    Find the path to a route JSON file in GCS by route ID.

    Args:
        bucket_name: Name of the GCS bucket
        route_id: Route ID to search for
        prefix: Prefix to search within (default: "all_routes/")

    Returns:
        Full path to the route JSON file in GCS
    """
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # List all blobs with the prefix
    blobs = bucket.list_blobs(prefix=prefix)

    # Search for file matching route_id pattern
    pattern = f"route_{route_id}.json"

    for blob in blobs:
        if blob.name.endswith(pattern):
            return blob.name

    raise FileNotFoundError(f"Route {route_id} not found in bucket {bucket_name} with prefix {prefix}")


def load_route_json_from_gcs(bucket_name: str, path: str, token: str = "google_default") -> Dict[str, Any]:
    """
    Load route JSON from GCS.

    Args:
        bucket_name: Name of the GCS bucket
        path: Path to the JSON file within the bucket
        token: Authentication token (default: "google_default")

    Returns:
        Dictionary containing the route JSON data
    """
    gcs_path = f"gs://{bucket_name}/{path}"

    with fsspec.open(gcs_path, 'r', token=token) as f:
        route_json = json.load(f)

    return route_json


def load_route_by_id(route_id: int, bucket_name: str = "cycle_more_bucket") -> Dict[str, Any]:
    """
    Load route JSON by route ID (convenience function combining find and load).

    Args:
        route_id: Route ID to load
        bucket_name: GCS bucket name (default: "cycle_more_bucket")

    Returns:
        Dictionary containing the route JSON data
    """
    print(f"Searching for route {route_id}...")
    path = find_route_in_gcs(bucket_name, route_id)
    print(f"Found at: {path}")

    route_json = load_route_json_from_gcs(bucket_name, path)
    print(f"Loaded route {route_id} successfully")

    return route_json

## Function 2: Extract Coordinates from Route JSON

In [4]:
def extract_coords(route_json: Dict[str, Any]) -> Tuple[List[Tuple[float, float]], List[float]]:
    """
    Extract (lat, lon) coordinates and elevation data from route JSON.

    Handles both FeatureCollection format and direct geometry format.

    Args:
        route_json: Route JSON dictionary

    Returns:
        Tuple of (coords_list, elevations_list)
        - coords_list: List of (lat, lon) tuples
        - elevations_list: List of elevation values in meters
    """
    coords = []
    elevations = []

    # Handle FeatureCollection format
    if route_json.get('type') == 'FeatureCollection':
        features = route_json.get('features', [])
        for feature in features:
            geometry = feature.get('geometry', {})
            geom_type = geometry.get('type')
            coordinates = geometry.get('coordinates', [])

            if geom_type == 'LineString':
                for coord in coordinates:
                    lon, lat = coord[0], coord[1]
                    coords.append((lat, lon))
                    if len(coord) > 2:
                        elevations.append(coord[2])

            elif geom_type == 'MultiLineString':
                for line in coordinates:
                    for coord in line:
                        lon, lat = coord[0], coord[1]
                        coords.append((lat, lon))
                        if len(coord) > 2:
                            elevations.append(coord[2])

    # Handle Overpass/relation format with members
    elif 'members' in route_json:
        for member in route_json['members']:
            if 'geometry' in member:
                for point in member['geometry']:
                    lat = point.get('lat')
                    lon = point.get('lon')
                    if lat is not None and lon is not None:
                        coords.append((lat, lon))
                        # Note: Overpass format typically doesn't include elevation
                        # Elevation would need to be fetched separately

    # Handle direct geometry format
    elif 'geometry' in route_json:
        geometry = route_json['geometry']
        geom_type = geometry.get('type')
        coordinates = geometry.get('coordinates', [])

        if geom_type == 'LineString':
            for coord in coordinates:
                lon, lat = coord[0], coord[1]
                coords.append((lat, lon))
                if len(coord) > 2:
                    elevations.append(coord[2])

        elif geom_type == 'MultiLineString':
            for line in coordinates:
                for coord in line:
                    lon, lat = coord[0], coord[1]
                    coords.append((lat, lon))
                    if len(coord) > 2:
                        elevations.append(coord[2])

    return coords, elevations

## Function 3: Plot Single Route on Map

In [5]:
def plot_single_route(coords: List[Tuple[float, float]], route_name: str = "Route") -> folium.Map:
    """
    Create an interactive Folium map with a single route.

    Args:
        coords: List of (lat, lon) tuples
        route_name: Name of the route for the popup

    Returns:
        folium.Map object
    """
    if not coords:
        raise ValueError("No coordinates provided")

    # Calculate center point
    center_lat = sum(coord[0] for coord in coords) / len(coords)
    center_lon = sum(coord[1] for coord in coords) / len(coords)

    # Create map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

    # Add route polyline
    folium.PolyLine(
        coords,
        color='blue',
        weight=4,
        opacity=0.8,
        popup=route_name
    ).add_to(m)

    # Add start marker (green)
    folium.Marker(
        coords[0],
        popup='Start',
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)

    # Add end marker (red)
    folium.Marker(
        coords[-1],
        popup='End',
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(m)

    return m

## Function 4: Plot Multiple Routes on Same Map

In [6]:
def plot_multiple_routes(
    route_data: List[Dict[str, Any]],
    bucket_name: str = "cycle_more_bucket"
) -> folium.Map:
    """
    Plot multiple recommended routes on the same map with different colors.

    Args:
        route_data: List of route dictionaries with 'route_id' and optionally 'route_name', 'distance_m'
        bucket_name: GCS bucket name

    Returns:
        folium.Map object with all routes displayed
    """
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'darkblue', 'darkred', 'darkgreen']

    all_coords = []
    route_coords_list = []

    # Load all routes and collect coordinates
    for idx, route in enumerate(route_data):
        route_id = route['route_id']

        try:
            # Load route JSON
            route_json = load_route_by_id(route_id, bucket_name)

            # Extract coordinates
            coords, _ = extract_coords(route_json)

            if coords:
                route_coords_list.append({
                    'coords': coords,
                    'route_id': route_id,
                    'route_name': route.get('route_name', f'Route {route_id}'),
                    'distance_m': route.get('distance_m', 'N/A'),
                    'color': colors[idx % len(colors)]
                })
                all_coords.extend(coords)

        except Exception as e:
            print(f"Error loading route {route_id}: {e}")

    if not all_coords:
        raise ValueError("No valid routes could be loaded")

    # Calculate center of all routes
    center_lat = sum(coord[0] for coord in all_coords) / len(all_coords)
    center_lon = sum(coord[1] for coord in all_coords) / len(all_coords)

    # Create map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

    # Add each route to the map
    for route in route_coords_list:
        distance_km = route['distance_m'] / 1000 if isinstance(route['distance_m'], (int, float)) else 'N/A'
        popup_text = f"{route['route_name']}<br>ID: {route['route_id']}<br>Distance: {distance_km} km"

        folium.PolyLine(
            route['coords'],
            color=route['color'],
            weight=4,
            opacity=0.7,
            popup=popup_text
        ).add_to(m)

        # Add start marker for each route
        folium.CircleMarker(
            route['coords'][0],
            radius=5,
            color=route['color'],
            fill=True,
            popup=f"Start: {route['route_name']}"
        ).add_to(m)

    return m

## Function 5: Main Function - Visualize Recommendations

In [7]:
def visualize_recommendations(
    recommendations: List[Dict[str, Any]],
    display_mode: str = "multiple",
    bucket_name: str = "cycle_more_bucket"
) -> folium.Map:
    """
    Main function to visualize route recommendations.

    Args:
        recommendations: List of route dictionaries from recommendation engine
                        Each should have 'route_id' and optionally 'route_name', 'distance_m', etc.
        display_mode: "single" or "multiple" (default: "multiple")
        bucket_name: GCS bucket name

    Returns:
        folium.Map object
    """
    if display_mode == "single":
        # Display only first recommendation
        route_id = recommendations[0]['route_id']
        route_json = load_route_by_id(route_id, bucket_name)
        coords, _ = extract_coords(route_json)
        route_name = recommendations[0].get('route_name', f'Route {route_id}')
        return plot_single_route(coords, route_name)

    else:  # multiple
        return plot_multiple_routes(recommendations, bucket_name)

## Test with Sample Route IDs

Using the route IDs from your screenshot:
- 14349467 - Ciclovia Pedemontana Alpina
- 16478126 - Dal Lago di Garda a Venezia
- 12688705 - Unnamed route
- 9876543 - Scenic River Loop (example)
- etc.

In [8]:
# Sample recommendations from your screenshot
sample_recommendations = [
    {
        "route_id": 14349467,
        "route_name": "Ciclovia Pedemontana Alpina",
        "distance_m": 89292.3,
        "ascent_m": 1354,
        "duration_s": 18702.4,
        "turn_density": 2.0158513080731,
        "similarity_score": 0.034958531949598
    },
    {
        "route_id": 16478126,
        "route_name": "Dal Lago di Garda a Venezia (Alternative, escursioni e collegamenti)",
        "distance_m": 156841.6,
        "ascent_m": 1554.8,
        "duration_s": 32059.8,
        "turn_density": 1.6449717421908474,
        "similarity_score": 0.037978063359428
    },
    {
        "route_id": 12688705,
        "route_name": "Unnamed route",
        "distance_m": 106648.6,
        "ascent_m": 1012.4,
        "duration_s": 21740.6,
        "turn_density": 1.622215331102909,
        "similarity_score": 0.032101170918636
    }
]

print("Sample recommendations loaded:")
for rec in sample_recommendations:
    print(f"  - {rec['route_name']} (ID: {rec['route_id']})")

Sample recommendations loaded:
  - Ciclovia Pedemontana Alpina (ID: 14349467)
  - Dal Lago di Garda a Venezia (Alternative, escursioni e collegamenti) (ID: 16478126)
  - Unnamed route (ID: 12688705)


## Test 1: Load Single Route by ID

In [9]:
# Test loading a single route
test_route_id = 14349467
route_json = load_route_by_id(test_route_id)

print(f"\nRoute JSON keys: {route_json.keys()}")
print(f"Route type: {route_json.get('type')}")
print(f"Route ID: {route_json.get('id')}")

Searching for route 14349467...
Found at: all_routes/tile_ita_west_route_14349467.json
Loaded route 14349467 successfully

Route JSON keys: dict_keys(['type', 'bbox', 'features', 'metadata'])
Route type: FeatureCollection
Route ID: None


## Test 2: Extract Coordinates

In [10]:
# Extract coordinates from the loaded route
coords, elevations = extract_coords(route_json)

print(f"\nTotal coordinates: {len(coords)}")
print(f"First coordinate: {coords[0]}")
print(f"Last coordinate: {coords[-1]}")
print(f"Elevations available: {len(elevations) > 0}")


Total coordinates: 2219
First coordinate: (45.467433, 7.882521)
Last coordinate: (45.034768, 7.522763)
Elevations available: True


## Test 3: Display Single Route on Map

In [11]:
# Display the single route
map_single = plot_single_route(coords, "Ciclovia Pedemontana Alpina")
map_single

## Test 4: Display Multiple Recommended Routes

In [12]:
# Display all recommended routes on one map
map_multiple = plot_multiple_routes(sample_recommendations)
map_multiple

Searching for route 14349467...
Found at: all_routes/tile_ita_west_route_14349467.json
Loaded route 14349467 successfully
Searching for route 16478126...
Found at: all_routes/tile_ita_west_route_16478126.json
Loaded route 16478126 successfully
Searching for route 12688705...
Found at: all_routes/belgium_route_12688705.json
Loaded route 12688705 successfully


## Test 5: Use Main Visualization Function

In [13]:
# Using the main function - multiple routes
map_viz = visualize_recommendations(sample_recommendations, display_mode="multiple")
map_viz

Searching for route 14349467...
Found at: all_routes/tile_ita_west_route_14349467.json
Loaded route 14349467 successfully
Searching for route 16478126...
Found at: all_routes/tile_ita_west_route_16478126.json
Loaded route 16478126 successfully
Searching for route 12688705...
Found at: all_routes/belgium_route_12688705.json
Loaded route 12688705 successfully


In [14]:
# Using the main function - single route
map_viz_single = visualize_recommendations(sample_recommendations, display_mode="single")
map_viz_single

Searching for route 14349467...
Found at: all_routes/tile_ita_west_route_14349467.json
Loaded route 14349467 successfully


## Summary of Functions

### Core Functions Created:

1. **`load_route_by_id(route_id, bucket_name)`**
   - Takes a route ID
   - Finds and loads the route JSON from GCS
   - Returns the route JSON dictionary

2. **`extract_coords(route_json)`**
   - Takes route JSON
   - Extracts (lat, lon) coordinates
   - Returns coordinate list and elevations

3. **`plot_single_route(coords, route_name)`**
   - Takes coordinates and route name
   - Creates Folium map with one route
   - Adds start/end markers

4. **`plot_multiple_routes(route_data, bucket_name)`**
   - Takes list of route dictionaries (with route_id)
   - Loads all routes from GCS
   - Displays all on one map with different colors

5. **`visualize_recommendations(recommendations, display_mode, bucket_name)`**
   - Main function to use in Streamlit
   - Takes recommendations from API
   - Returns interactive Folium map

### Next Steps:
- Test these functions with your actual route IDs
- Once working, integrate into Streamlit app
- Replace the placeholder map in app.py with real route visualization